In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip

In [6]:
!unzip -o -j '../Datasets ML/moviedataset.zip' 

Archive:  ../Datasets ML/moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [27]:
import requests
url='https://gist.githubusercontent.com/deshwal88/33815a3ac654dc9834d96fcffc54f6c2/raw/41c4786a6e37cf6244576a0daa1af292ecfbbd36/tags.csv'
r=requests.get(url)

<Response [200]>

In [49]:
import io
pd.read_csv(io.StringIO(r.content.decode('utf-8')))

,userId,movieId,tag,timestamp
0,19,2324,bittersweet,1428651158
1,19,2324,holocaust,1428651112
2,19,2324,World War II,1428651118
3,23,7075,hilarious,1378675786
4,23,7075,Underrated,1378675786
...,...,...,...,...
586989,247751,96655,future,1366041126
586990,247751,96655,robots,1366041128
586991,247753,356,classic,1430438776
586992,247753,356,oscar (best supporting actor),1430438776


In [9]:
    
movies=pd.read_csv('../ml-latest/movies.csv')

movies['genres']=movies.genres.str.split('|')
movies['years']=movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['years']=movies.years.str.extract('(\d\d\d\d)',expand=False)
movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')
movies['title']=movies.title.str.strip()

new=movies.copy()
for i,row in new.iterrows():
    for genre in row['genres']:
        new.loc[i,genre]=1
new=new.fillna(0)

rating=pd.read_csv('../ml-latest/ratings.csv')
rating=rating.drop(['timestamp','userId'],1)
rating.describe()

user_input=[{'title':'Inception','rating':4},
            {'title':'Batman Begins','rating':5},
            {'title':'Toy Story','rating':3},
            {'title':'Star Trek: Generations','rating':2.5},
            {'title':'Super Mario Bros.','rating':2}]
user_input=pd.DataFrame(user_input)

input_mov=movies[movies['title'].isin(user_input['title'].tolist())]
input_mov.drop(['genres','years'],1,inplace=True)
input_mov.reset_index()
final=pd.merge(user_input,input_mov)
final

genre_rate=new[new['title'].isin(final.title)]

In [10]:
genre_rate=genre_rate.reset_index().drop('index',axis=1)
genre_rate=genre_rate.drop(['movieId','title','genres','years'],1)
genre_rate=genre_rate.transpose()
rating=genre_rate.dot(final['rating'])

new

,movieId,title,genres,years,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,151697,Grand Slam,[Thriller],1967,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34204,151701,Bloodmoney,[(no genres listed)],2010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34205,151703,The Butterfly Circus,[Drama],2009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34206,151709,Zero,"[Drama, Sci-Fi]",2015,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
genresinfo=new.set_index(new['movieId'])
genresinfo=genresinfo.drop(['movieId','title','years','genres'],1)
recommend=(genresinfo.dot(rating))/rating.sum()

d=recommend.sort_values(ascending=False)

In [14]:
movies[movies.movieId.isin(d[0:10].index)]

,movieId,title,genres,years
542,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11785,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
11806,52462,Aqua Teen Hunger Force Colon Movie Film for Th...,"[Action, Adventure, Animation, Comedy, Fantasy...",2007
12021,54278,Underdog,"[Action, Adventure, Children, Comedy, Fantasy,...",2007
16884,85261,Mars Needs Moms,"[Action, Adventure, Animation, Children, Comed...",2011
30066,136618,Pokémon the Movie: Genesect and the Legend Awa...,"[Action, Adventure, Animation, Children, Fanta...",2013
32313,144350,Under the Mountain,"[Action, Adventure, Children, Drama, Fantasy, ...",2009
33509,148775,Wizards of Waverly Place: The Movie,"[Adventure, Children, Comedy, Drama, Fantasy, ...",2009


In [16]:
d

movieId
85261     0.697987
5018      0.691275
32031     0.691275
546       0.677852
144350    0.677852
            ...   
134627    0.000000
59188     0.000000
59295     0.000000
134601    0.000000
151711    0.000000
Length: 34208, dtype: float64